# S3 Minio Tutorial

## Python Package Requirements

1. [s3fs](https://pypi.org/project/s3fs/) - สำหรับเชื่อมต่อ S3 API ของ Minio และ AWS S3
2. [configobj](https://pypi.org/project/configobj/) -  สำหรับอ่าน file `.ini` 

    Pip install:
    ```bash
    pip install s3fs configobj

    ```


 [s3fs Documentation: https://s3fs.readthedocs.io](https://s3fs.readthedocs.io/en/latest/index.html)

In [ ]:
import os

from configobj import ConfigObj
from s3fs import S3FileSystem

## Read Configfile `config.ini`

Example `config.ini`
```ini
[minio]
endpoint_url = https://127.0.0.1:9000
key = minioadmin
secret = minioadmin
bucket=bucket_name
path=path_prefix
```

In [ ]:
conf: dict = ConfigObj('config.ini', raise_errors=True)
conf.keys()

## Connect MinIO


In [ ]:
minioConf = conf["minio"]

s3: S3FileSystem = S3FileSystem(
    anon=False,
    key=minioConf["key"],
    secret=minioConf["secret"],
    endpoint_url=minioConf["endpoint_url"], 
) 

# check can connect to minio bucket `name`
print("can connect minio bucket: ", s3.ls("/"))
print("connected to minio.")

## List file in MinIO


```python
bucket = "bucket_name"
path_prefix = "path/to/"
s3.ls(f"{bucket}/{path_prefix}")

```

In [ ]:
bucket = conf["minio"]["bucket"]
path_prefix = conf["minio"]["path"]

# list files in bucket/path
s3.ls(f"{bucket}/{path_prefix}")

## Upload file to MinIO

In [ ]:
s3_filepath:str = f"{bucket}/{path_prefix}/test.txt"
local_filepath: str = "storage/test.txt"


# check file exists
if s3.exists(s3_filepath):
    print(f"{s3_filepath} - file exists ")
    print("Overwrite if exists.")



# upload file (Overwrite if exists)
print("Upload file: ", local_filepath)
s3.put(local_filepath, s3_filepath)

# show flie upload
s3.ls(s3_filepath)

print("file uploaded.")


## Read file from MinIO

In [ ]:
s3_filepath:str = f"{bucket}/{path_prefix}/test.txt"
if s3.exists(s3_filepath):
    print("Open file")
    with s3.open(s3_filepath, mode="r") as f:
        print(f.read())
        pass

## Download file from MinIO


In [ ]:
s3_filepath:str = f"{bucket}/{path_prefix}/test.txt"
download_file: str = f"{local_filepath}.download"
s3.download(s3_filepath, download_file)

with open(download_file, mode="r") as f:
    print(f.read())
    pass

os.unlink(download_file)

## Remove file In MinIO

In [ ]:
remove_file: str =  f"{bucket}/{path_prefix}/test.txt"
s3.rm_file(remove_file)

if not s3.exists(remove_file):
    print(f"{remove_file} - file removed.")